## Описание данных

Проект состоит из двух частей: в первой вы решите несколько задач в SQL-тренажёре, а во второй — напишите SQL-запросы в Jupyter Notebook с помощью библиотеки SQLAlchemy. Задачи второй части проверят вручную.
В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили. 


### Таблица badges

Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 

Поле	Описание

id	Идентификатор значка, первичный ключ таблицы

name	Название значка

user_id	Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users

creation_date	Дата присвоения значка


### Таблица post_types

Содержит информацию о типе постов. Их может быть два:

•	Question — пост с вопросом;

•	Answer — пост с ответом.

Поле	Описание

id	Идентификатор поста, первичный ключ таблицы

type	Тип поста


### Таблица posts

Содержит информацию о постах.

Поле	Описание

id	Идентификатор поста, первичный ключ таблицы

title	Заголовок поста

creation_date	Дата создания поста

favorites_count	Число, которое показывает, сколько раз пост добавили в «Закладки»

last_activity_date	Дата последнего действия в посте, например комментария

last_edit_date	Дата последнего изменения поста

user_id	Идентификатор пользователя, который создал пост, внешний ключ к таблице users

parent_id	Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом

post_type_id	Идентификатор типа поста, внешний ключ к таблице post_types

score	Количество очков, которое набрал пост

views_count	Количество просмотров


### Таблица users

Содержит информацию о пользователях.

Поле	Описание

id	Идентификатор пользователя, первичный ключ таблицы

creation_date	Дата регистрации пользователя

display_name	Имя пользователя

last_access_date	Дата последнего входа

location	Местоположение

reputation	Очки репутации, которые получают за хорошие вопросы и полезные ответы

views	Число просмотров профиля пользователя


### Таблица vote_types

Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 

•	UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.

•	DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.

•	Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.

•	Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.

•	Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.

Поле	Описание

id	Идентификатор типа голоса, первичный ключ

name	Название метки


### Таблица votes

Поле                                                  Описание

Id                     Идентификатор голоса, первичный ключ

post_id            Идентификатор поста, внешний ключ к таблице posts

user_id    Идентификатор пользователя, который поставил посту голос, внешний    ключ к таблице users

bounty_amount    Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту

vote_type_id     Идентификатор типа голоса, внешний ключ к таблице vote_types

creation_date      Дата назначения голоса


1. Найдите количество вопросов, которые набрали более 300 очков или как минимум 100 раз были добавлены в "Закладки".

In [ ]:
SELECT COUNT(pt.id) AS cnt
FROM stackoverflow.post_types pt
JOIN stackoverflow.posts p ON pt.id=p.post_type_id
WHERE pt.type = 'Question'
      AND p.score>300 OR p.favorites_count>=100;

2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(q.count),0)
FROM
(SELECT COUNT(id),
        creation_date::date
FROM stackoverflow.posts 
WHERE post_type_id =1
GROUP BY creation_date::date
HAVING creation_date::date BETWEEN '2008-11-01' AND '2008-11-18') AS q;

3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT b.user_id)
FROM stackoverflow.badges b
JOIN stackoverflow.users u ON u.id=b.user_id
WHERE u.creation_date::date = b.creation_date::date;

4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(pt.id)
FROM
(SELECT p.id
FROM stackoverflow.posts p
JOIN stackoverflow.votes v ON p.id = v.post_id
JOIN stackoverflow.users u ON p.user_id = u.id
WHERE u.display_name LIKE 'Joel Coehoorn' AND v.id>0
GROUP BY p.id) AS pt;

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       ROW_NUMBER() OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT *
FROM (SELECT v.user_id,
             COUNT(vt.id) AS vt_cnt
     FROM stackoverflow.votes v
     JOIN stackoverflow.vote_types vt ON vt.id=v.vote_type_id
     WHERE vt.name='Close' 
     GROUP BY v.user_id
     ORDER BY vt_cnt DESC
     LIMIT 10) AS uv
ORDER BY uv.vt_cnt DESC, uv.user_id DESC; 

7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей:

идентификатор пользователя;

число значков;

место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT *,
       DENSE_RANK() OVER(ORDER BY b.b_cnt DESC) AS rating
FROM (SELECT user_id,
             COUNT(id) AS b_cnt
     FROM stackoverflow.badges
     WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
     GROUP BY user_id
     ORDER BY b_cnt DESC, user_id
     LIMIT 10) AS b

8. Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:

заголовок поста;

идентификатор пользователя;

число очков поста;

среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
WITH cs AS
(SELECT ROUND(AVG(score),0) AS avg_score,
        user_id
FROM stackoverflow.posts 
WHERE score <> 0 AND title IS NOT NULL
GROUP BY user_id)
SELECT p.title,
       cs.user_id,
       p.score,
       cs.avg_score
FROM cs
JOIN stackoverflow.posts p ON cs.user_id=p.user_id              
WHERE p.score <> 0 AND p.title IS NOT NULL;

9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE user_id IN(SELECT user_id
                FROM stackoverflow.badges
                GROUP BY user_id
                HAVING COUNT(id)>1000)
               AND title IS NOT NULL;

10. Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

пользователям с числом просмотров больше либо равным 350 присвойте группу 1;

пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;

пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE 
          WHEN views >= 350 THEN 1
          WHEN views < 100 THEN 3
          ELSE 2
       END AS group
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views >0;

11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH grp AS (SELECT g.id,
                      g.views,
                      g.group,
                      MAX(g.views) OVER(PARTITION BY g.group) AS max
               FROM (SELECT id,
                            views,
                            CASE 
                               WHEN views >= 350 THEN 1
                               WHEN views < 100 THEN 3
                               ELSE 2
                           END AS group
                     FROM stackoverflow.users
                     WHERE location LIKE '%Canada%' AND views >0) AS g)
SELECT grp.id,
       grp.group,
       grp.max
FROM grp
WHERE grp.views=grp.max
ORDER BY grp.views DESC,grp.id;

12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

номер дня;

число пользователей, зарегистрированных в этот день;

сумму пользователей с накоплением.

In [ ]:
SELECT *,
       SUM(cu.cnt_us) OVER(ORDER BY cu.days) AS cnt_users
FROM
(SELECT EXTRACT(DAY FROM creation_date::date) AS days,
       COUNT(id) AS cnt_us
FROM stackoverflow.users
WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY EXTRACT(DAY FROM creation_date::date)) AS cu;

13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:

идентификатор пользователя;

разницу во времени между регистрацией и первым постом.

In [ ]:
WITH dt AS (SELECT DISTINCT user_id,
                   MIN(creation_date) OVER(PARTITION BY user_id) AS min_dt
           FROM stackoverflow.posts)
SELECT dt.user_id,
       (dt.min_dt - u.creation_date) AS dif
FROM stackoverflow.users AS u
JOIN dt ON u.id=dt.user_id;

В этой части проекта вам нужно написать несколько SQL-запросов в Jupyter Notebook. Эти задания проверят вручную, и вы получите комментарии к составленным запросам. 

Не забудьте подключиться к базе с помощью SQLAlchemy. Вспомните инструкцию из урока 11 «Когортный анализ. Представление результатов» в теме 4 «Установка и настройка базы данных и клиента базы данных». Необходимые данные находятся в таблицах схемы stackoverflow.  

Некоторые задания включают дополнительные вопросы — не пропустите их. На часть вопросов можно ответить текстом, а для некоторых понадобится визуализация. Помните, что результат запроса можно выгрузить в датафрейм. 

Чтобы ожидаемый результат было легче представить, мы добавили к каждому заданию небольшой фрагмент итоговой таблицы. В запросах вы можете использовать любые подходящие названия полей.